<header>Archery Capstone</header>

This capstone project was created in support of the CodeYou program to showcase the materials covered in the Data Analysis pathway. 
The code blocks below will walk you through the following:
    1.  Accessing three layers created within ArcGIS Online to collect archery related stats from 3D target events
    2.  Pulling these layers down to data frames to work with separately from the authoritative data within ArcGIS Online
    3.  Cleaning and joining the data so it can be used for futher analysis and graphic creation
    4.  Creating graphics and shot summaries for the archer to reference and analyze for trends

Import Modules

In [ ]:
from arcgis.gis import GIS
from arcgis.features import FeatureLayer
import pandas as pd
import pandasql as psql

Define Variables

Download AGOL layers to data frames

In [ ]:
# Connect to ArcGIS Online
gis = GIS("https://www.arcgis.com")

# Search for the layers by title
layer1_item = gis.content.search("title:YourLayerTitle1", item_type="Feature Layer", max_items=1)[0]
layer2_item = gis.content.search("title:YourLayerTitle2", item_type="Feature Layer", max_items=1)[0]
layer3_item = gis.content.search("title:YourLayerTitle3", item_type="Feature Layer", max_items=1)[0]

# Access the first layer in each item
feature_layer1 = layer1_item.layers[0]
feature_layer2 = layer2_item.layers[0]
feature_layer3 = layer3_item.layers[0]

# Query the layers to get all features
features1 = feature_layer1.query(where="1=1")
features2 = feature_layer2.query(where="1=1")
features3 = feature_layer3.query(where="1=1")

# Convert features to pandas DataFrames
df1 = features1.sdf
df2 = features2.sdf
df3 = features3.sdf

# Print the DataFrames
print(df1.head())
print(df2.head())
print(df3.head())

<header>Clean data frames using SQL</header>

Populate fields
Score to Scored Ring
Adjustment Made to No if null

In [ ]:
# Example SQL query to select specific columns and filter rows
query = """
SELECT column1, column2
FROM df1
WHERE column3 > 100
"""
result_df = psql.sqldf(query, locals())

# Print the result of the SQL query
print(result_df)

Clean data frames using SQL
    2.  Create flag fields on multi select questions
        Adjustment
            Increase_Yardage, Decrease_Yardage, Sight_Left, Sight_Right
        Position
            Quartered_to, Quartered_From, Broadside, Leaning_Forward, Leaning_Back
        Shot Placement
            On_Aim_Point, High, Low, Left, Right, Miss, Kick_Out
        Shot Execution
            Unsure, Felt_Good, Bad_Shoulder, Bad_Release, Movement_in_Shot, Unable_to_Center_Bubble, Dip, Push_Up, Glare, other
        Release Details
            Too_Soon, Fast, Slow, Off_Face, Hand_Moved
        Weather Conditions Present
        Personal Conditions

Clean data frames using SQL
    3.  Create a new layer by doing a join on t1.Animals (t1.TargetName = t3.target) and t2.shooting conditions (t3.date_and_time = t2.date_and_time_of_Shoot) to t3.shot report out

Create graphics
    Shot placement on color
    Shot placement on aim point
    Shot Placement on range
    Shot placement on 
    Animal Size on color
    Shot placement on position
    Overall Shot Placement
    Release Details on scored ring
    

Create Readme
    Purpose
    Background
    Goals
    Modules Needed
    

Future Work
    1.  Identify further parameters for performing analysis
    2.  Fine tune the shoot report mobile survey to collect additional informaiton (i.e. which direction is the target facing), populate defaults, and improve the data entry process, and allow for open ranges. 
    3.  Create a PDF or Word Report for daily shoot summaries
    
    